<h1><center>第5章 合并</center></h1>

In [1]:
import pandas as pd
import numpy as np

### 一、用append添加行

In [7]:
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'), index=list('ab'))
s = pd.Series(['new_1','new_2'],index=list('AB'))
s.name = 'new_s'
dicts = [{'A': 'newa1', 'B': 'newb1'},
         {'A': 5, 'B': 6}]
df.append(dicts)

,A,B
a,1,2
b,3,4
0,newa1,newb1
1,5,6


In [2]:
# 函数用法：df.append(other,ignore_index=False,verify_integrity=False,sort=False)
# (1) 基础用法：
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'), index=list('ab'))
# a.用Series添加行：
s = pd.Series(['new_1','new_2'],index=list('AB'))
#df.append(s) # 报错，因为Series没有加name
s.name = 'new_s'
df.append(s)
# b.用dict添加行：
dicts = [{'A': 'newa1', 'B': 'newb1'},
         {'A': 5, 'B': 6}]
df.append(dicts)
# (2) 添加DataFrame：
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'), index=list('ab'))
df.append(df2)
#df.append(df2,ignore_index=True) # 重新创建默认索引
#df.append(df2,verify_integrity=True) # 报错，因为索引不唯一
# 另一个例子：
df = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'E': ['E2', 'E3', 'E6', 'E7']},
                     index=[2, 3, 6, 7])
df2 = pd.DataFrame({'A': ['A1', 'A2', 'A3', 'A5'],
                    'D': ['D1', 'D2', 'D3', 'D5'],
                    'E': ['E1', 'E2', 'E3', 'E5']},
                     index=[1, 2, 3, 5])
df.append(df2, sort=True).head() # 对列进行排序

,A,B,D,E
2,NaN,B2,D2,E2
3,NaN,B3,D3,E3
6,NaN,B6,D6,E6
7,NaN,B7,D7,E7
1,A1,NaN,D1,E1


### 二、用assign添加列

In [3]:
# 基本用法：
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df.assign(new_col1=df['A']*df['B'])
# Python-3.6+的版本中，能为表格通过assign一次添加多个列：
df.assign(col1=lambda x:df['A']+df['B'],
          col2=lambda x:df['A']-df['B'],)

,A,B,col1,col2
0,1,2,3,-1
1,3,4,7,-1


### 三、用combine与update填充

In [4]:
# 函数用法：df.combine(other,func,fill_value=None,overwrite=True)
# (1)直接组合：
#df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
#df2 = pd.DataFrame({'A': [8, 7], 'B': [6, 5]})
#df1.combine(df2,lambda x,y:print(x,y))
# 从上面可以看出，combine方法是对表的列分别循环的
# 例子1
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [8, 7], 'B': [6, 5]})
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)
# 例子2
df2 = pd.DataFrame({'B': [8, 7], 'C': [6, 5]},index=[1,2])
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)
# 例子3
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,overwrite=False) 
# overwrite设置为True代表后面的表没有的行列都会设置为NaN
# 例子4
# df1.combine(df2,lambda x,y:print(x),fill_value=-1)
# 如果添加了fill_value那么df1就在新增匹配df2的元素位置填充-1
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=-1)
# (2)特别的combine_first方法：
#主要作用是用df2填补df1的缺失值
# 例子1
df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)
# 例子2
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
df1.combine_first(df2)

,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


In [5]:
# update用法：df.update(other, join='left', overwrite=True, filter_func=None, errors='ignore')
# 具备两个特点：①返回的框只会与df大小一致②第二个框中的nan元素不会起作用③没有返回值，直接在df上操作
# 例子1
df = pd.DataFrame({'A': [1, 2, 3],
                   'B': [400, 500, 600]})
new_df = pd.DataFrame({'B': [4, 5, 6],
                       'C': [7, 8, 9]})
df.update(new_df)
df
# 例子2
df = pd.DataFrame({'A': ['a', 'b', 'c'],
                   'B': ['x', 'y', 'z']})
new_df = pd.DataFrame({'B': ['d', 'e']}, index=[1, 2])
df.update(new_df)
df
# 例子3
df = pd.DataFrame({'A': [1, 2, 3],
                   'B': [400, 500, 600]})
new_df = pd.DataFrame({'B': [4, np.nan, 6]})
df.update(new_df)
df

,A,B
0,1,4.0
1,2,500.0
2,3,6.0


### 四、用concat连接

In [93]:
#pd.concat(objs, axis=0, join='outer', ignore_index=False, keys=None,
#                                    sort=None, levels=None, names=None, verify_integrity=False, copy=True)
#(1)简单拼接：
df1 = pd.DataFrame({'A': ['A0', 'A1'],
                    'B': ['B0', 'B1']},
                    index = [0,1])
df2 = pd.DataFrame({'A': ['A2', 'A3'],
                    'B': ['B2', 'B3']},
                    index = [2,3])
df3 = pd.DataFrame({'A': ['A1', 'A3'],
                    'D': ['D1', 'D3'],
                    'E': ['E1', 'E3']},
                    index = [1,3])
pd.concat([df1,df2])
#(2)axis=1时沿列方向拼接：
pd.concat([df1,df2],axis=1)
#(3)join='outer'时取拼接方向的并集，'inner'时取拼接方向（默认为纵向拼接，因此为列交集）的交集
pd.concat([df3,df1],join='inner',sort=True)
#sort不写时会报告警告，因为在pandas的下一个版本中会设置sort为False但是当下为True
pd.concat([df3,df1],join='outer',sort=True)
#(4)ignore_index设置为True时采取默认索引
pd.concat([df3,df1],ignore_index=True,sort=True)
#(5)verify_integrity检查是否列是否唯一
#pd.concat([df3,df1],verify_integrity=True,sort=True) # 报错
#(6)添加Series
s = pd.Series(['X0', 'X1'], name='X')
pd.concat([df1,s],axis=1)
#(7)keys与levels的使用
#首先看keys：
#keys的作用是对凭借的块以键命名，可以调用：
pd.concat([df1,df2], keys=['x', 'y'])
#pd.concat([df1,df2], keys=['x', 'y']).index # 此时直接变成了多级索引
df_k = pd.concat({'x': df1, 'y': df2}) # 利用字典建立keys
df_k.loc['x']
df_k.loc[('x',0)]
#然后是levels：
#这个参数在官方doc上给出了一个令人有些费解的例子：
pd.concat([df1,df2], keys=['x', 'y'],
                  levels=[['x', 'y', 'z']])
#为什么同一层的levels会超过keys呢？

A   B
x 0  A0  B0
  1  A1  B1
y 2  A2  B2
  3  A3  B3

#### 查阅[相关资料](https://stackoverflow.com/questions/49620538/what-are-the-levels-keys-and-names-arguments-for-in-pandas-concat-functio). 可以找到解答：设我们现有3个分类变量，但是只有两张表，当分组操作的时候希望3个都能够信息汇总，此时可以借助分类变量

In [54]:
df = pd.concat([df1,df2], keys=['x', 'y'],levels=[['x', 'y', 'z']])
display(df.groupby(level=0).sum())
cats = ['x','y','z']
lvl = pd.CategoricalIndex(cats, categories=cats, ordered=True)
df = pd.concat([df1,df2], keys=['x', 'y'],levels=[lvl])
df.index.levels[0]
df.groupby(level=0).sum()

,A,B
x,A0A1,B0B1
y,A2A3,B2B3


,A,B
x,A0A1,B0B1
y,A2A3,B2B3
z,0,0


### 五、用merge与join连接

In [106]:
#merge函数：
#pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         #left_index=False, right_index=False, sort=True,
         #suffixes=('_x', '_y'), copy=True, indicator=False,
         #validate=None)
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']}) 
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3']})
#(1)一组键
pd.merge(left, right, on='key1')
#相同的列名别强制加上suffixes
#(2)多组键
pd.merge(left, right, on=['key1','key2'])
#默认使用inner连接，因为是默认横向拼接，所以取行向上keys的交集
pd.merge(left, right, how='outer', on=['key1','key2'])
#左连接是指以第一个表索引为基准，右边的表中如果不再左边的则不加入，如果在左边的就以笛卡尔积的方式加入
pd.merge(left, right, how='left', on=['key1', 'key2'])
pd.merge(left, right, how='right', on=['key1', 'key2'])
#下面的例子更加清晰地描述了笛卡尔积的方式：
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 2, 2]})
pd.merge(left, right, on='B', how='outer')
#(3)validate参数
#这个参数检验的是到底哪一边出现了重复索引，如果是“one_to_one”则两侧索引都是唯一，如果"one_to_many"则左侧唯一
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 3, 4]})
#pd.merge(left, right, on='B', how='outer',validate='one_to_one') #报错
left = pd.DataFrame({'A': [1, 2], 'B': [2, 1]})
pd.merge(left, right, on='B', how='outer',validate='one_to_one')
#(4)indicator参数
#这个参数指示了，合并后该行索引的来源
df1 = pd.DataFrame({'col1': [0, 1], 'col_left': ['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2], 'col_right': [2, 2, 2]})
pd.merge(df1, df2, on='col1', how='outer', indicator=True) #indicator='indicator_column'也是可以的

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


In [92]:
#join函数：
#df.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
#(1)简单合并：
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                    index=['K0', 'K2', 'K3'])
left.join(right)
#(2)many_to_one模式（用join往往更方便）
#单层key：
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                     index=['K0', 'K1'])
left.join(right, on='key')
#多层key：
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1']})
index = pd.MultiIndex.from_tuples([('K0', 'K0'), ('K1', 'K0'),
                                   ('K2', 'K0'), ('K2', 'K1')])
right = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']},
                     index=index)
left.join(right, on=['key1','key2']) #默认左连接

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


### 六、连接函数的异同

#### 这段总结来自于Theodore Petrou所著Pandas Cookbook中的368页：
| 方法   | 说明  |
| :----  | :----  |
| concat | 1.将多个pandas对象进行横向或纵向拼接 |
|  | 2.只用index对齐(没有on这个参数) |
|  | 3.默认outer连接，可选inner连接 |
| join  | 1.将多个pandas对象横向拼接 |
|   | 2.遇到重复的索引项时会使用笛卡尔积 | 
|   | 3.默认左连接，可选inner、outer、right连接 |
|   | 4.被调用的数据框列或索引与参数中的数据框索引对齐 |
| merge  | 1.将两个pandas对象横向拼接 |
|   | 2.遇到重复的索引项时会使用笛卡尔积 | 
|   | 3.默认inner连接，可选left、outer、right连接 |
|   | 4.被调用的数据框列或索引与参数中的数据框列或索引对齐 |

### 七、问题与练习
#### 1. 请思考什么是append/assign/combine/update/concat/merge/join各自最适合使用的场景，并举出相应的例子。
#### 2. merge_ordered和merge_asof的作用是什么？和merge是什么关系？
#### 3. 请构造一个多级索引与多级索引合并的例子，尝试使用不同的合并函数。

#### 【综合练习一】有2张公司的员工信息表，每个公司共有16名员工，共有五个公司，请解决如下问题：
#### (a) 每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。
#### (b) 将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。
#### (c) 现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，最后对于没有信息的员工，利用缺失值填充。
#### 【综合练习二】有2张课程的分数表（分数随机生成），但专业课（学科基础课、专业必修课、专业选修课）与其他课程混在一起，请解决如下问题：
#### (a) 将两张表分别拆分为专业课与非专业课（结果为四张表）。
#### (b) 将两张专业课的分数表和两张非专业课的分数表分别合并，请使用多种方法（如本章中提及的3种方式）。
#### (c) 不使用(a)中的步骤，请直接读取两张表合并后拆分。
#### (d) 专业课程中有缺失值吗，如果有的话请在完成(3)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。